<a href="https://colab.research.google.com/github/zeinabkobaissi/aiengineer_projects/blob/main/finetuning_exercise_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc https://www.dropbox.com/s/lkd0eklmi64m9xm/AirlineTweets.csv

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('AirlineTweets.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df=df[['airline_sentiment','text']]
df

In [ ]:
df['airline_sentiment'].hist()

In [ ]:
target_map = { 'positive': 1, 'negative': 0, 'neutral': 2}
df['target'] = df['airline_sentiment'].map(target_map)

In [ ]:
df1=df[['text','target']]
df1.columns=['sentence','label']
df1.to_csv('data.csv',index=False)


In [ ]:
df1

In [ ]:
!pip3 install datasets

In [ ]:
from datasets import load_dataset
raw_dataset=load_dataset('csv',data_files='data.csv')

In [ ]:
raw_dataset

In [ ]:
split=raw_dataset['train'].train_test_split(test_size=0.3,seed=42)

In [ ]:
split

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint='bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation = True)

In [ ]:
tokenized_dataset = split.map(tokenize_fn, batched = True)

In [ ]:
##  finetuning architecture

In [ ]:
from transformers import AutoModelForSequenceClassification,TrainingArguments,Trainer

In [ ]:
 model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=3)

In [ ]:
!pip3 install torchinfo

In [ ]:
from torchinfo import summary
summary(model)

In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
training_args = TrainingArguments(output_dir='training_dir',
                                  eval_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=3,
                                  report_to='none',
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=64,
                                  )

In [ ]:
import transformers
print(transformers.__version__)
from transformers import TrainingArguments
print(TrainingArguments.__init__.__code__.co_varnames)


In [ ]:
## metrics
def compute_metrics(logits_and_labels):
  logits,labels = logits_and_labels
  predictions=np.argmax(logits,axis=1)
  acc= np.mean(labels==predictions)
  f1= f1_score(predictions,labels,average='micro')
  return {'accuracy' : acc , 'f1score': f1}


In [ ]:
## trainer
trainer = Trainer(model,
                  training_args,
                  train_dataset = tokenized_dataset["train"],
                  eval_dataset = tokenized_dataset["test"],
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [ ]:
## start making predictions
from transformers import pipeline
saved_model=pipeline('text-classification',model='training_dir/checkpoint-1282')


In [ ]:
device = 0 if torch.cuda.is_available() else -1

In [ ]:
from transformers import pipeline
saved_model=pipeline('text-classification',model='training_dir/checkpoint-1282',device=device)


In [ ]:
predictions=saved_model(split['test']['sentence'][:10])

In [ ]:
predictions

In [ ]:
!pip install nbstripout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks"


In [ ]:
!ls

In [ ]:
!nbstripout finetuning-exercise.ipynb
